In [8]:
# Lire le fichier JSON et identifier la ligne problématique
with open('scrapped_content_from_referentiel.JSON', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Afficher la ligne 415 et quelques lignes autour pour identifier l'erreur
print("Ligne 415 et alentours :")
print(lines[414])  # Affiche la ligne 415 (les indices commencent à 0)
print(lines[413])  # Affiche la ligne avant la ligne 415
print(lines[415])  # Affiche la ligne après la ligne 415

# Corriger la virgule en trop dans la ligne 415 (si c'est la cause de l'erreur)
lines[414] = lines[414].rstrip(',\n')  # Retirer la virgule et les espaces/retour à la ligne à la fin de la ligne

# Enregistrer le fichier JSON corrigé
with open('scrapped_content_from_referentiel_corrected.JSON', 'w', encoding='utf-8') as file:
    file.writelines(lines)

# Vérification : charger le JSON corrigé
import json

try:
    with open('scrapped_content_from_referentiel_corrected.JSON', 'r', encoding='utf-8') as file:
        corrected_data = file.read()
    data = json.loads(corrected_data)  # Vérifier si le fichier est maintenant valide
    print("Le JSON a été chargé avec succès.")
    
    # Si le JSON est valide, convertir en DataFrame et exporter en CSV
    import pandas as pd
    df = pd.json_normalize(data)
    df.to_csv('converted_file.csv', index=False, encoding='utf-8')
    print("Le fichier CSV a été généré avec succès.")
except json.JSONDecodeError as e:
    print(f"Erreur de chargement JSON après correction : {e}")


Ligne 415 et alentours :
    },

        "compétences": "COMPÉTENCES DE BASE\nSAVOIR FAIRE\nContrôler la réception des commandes et établir l'ordre de traitement\nPlanifier l'activité des caristes et préparateurs de commandes, ... selon l'ordre de traitement des commandes\nOrganiser la répartition des emplacements de stockage sur le site et en suivre la gestion\nCoordonner et contrôler les opérations logistiques de réception, expédition et livraison\nOrganiser ou contrôler la gestion de stocks de produits, marchandises ou matières premières (référencement, rotation, ...)\nInformer les clients internes ou externes sur le traitement et le suivi des commandes\nRenseigner les supports de suivi d'activité et identifier les écarts\nSAVOIR\nCOMPÉTENCES SPÉCIFIQUES"

]
Le JSON a été chargé avec succès.
Le fichier CSV a été généré avec succès.


In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords

import nltk


In [17]:
df = pd.read_csv("ref.csv")
df.head()

,job_name,appellations,identité,compétences
0,Conception - développement produits d'assuranc...,Chargé / Chargée de produit en assurances\nChe...,DÉFINITION\nConçoit et adapte de nouveaux prod...,COMPÉTENCES DE BASE\nSAVOIR FAIRE\nAnalyser le...
1,Conseil clientèle en assurances (C1102),Agent commercial / Agente commerciale en assur...,DÉFINITION\nInforme et conseille des particuli...,COMPÉTENCES DE BASE\nSAVOIR FAIRE\nAnalyser le...
2,Courtage en assurances (C1103),Agent général / Agente générale d'assurances\n...,DÉFINITION\nAccompagne une clientèle (particul...,COMPÉTENCES DE BASE\nSAVOIR FAIRE\nMettre en o...
3,Direction d'exploitation en assurances (C1104),Chargé / Chargée d'agence en assurances\nCharg...,DÉFINITION\nOrganise la planification de l'act...,COMPÉTENCES DE BASE\nSAVOIR FAIRE\nDéfinir une...
4,Études actuarielles en assurances (C1105),Actuaire Confirmé / Sénior\nActuaire Junior\nC...,"DÉFINITION\nEvalue à court, moyen et long term...",COMPÉTENCES DE BASE\nSAVOIR FAIRE\nProposer de...


In [24]:
data_cleaned = df.copy()

In [25]:
data_cleaned = data_cleaned.rename(columns={'job_name': 'métier'})


In [26]:
# Télécharger les ressources nécessaires pour NLTK
nltk.download('stopwords')


# Charger les stopwords en français
stop_words = set(stopwords.words('french'))

def nettoyer_colonne_avance(texte):
    if not isinstance(texte, str):  # Vérifie que la valeur est une chaîne
        return ''
    # Convertir en minuscules
    texte = texte.lower()
    # Remplacer les caractères spéciaux et ponctuations par des espaces
    texte = re.sub(r'[^\w\s]', ' ', texte)
    # Supprimer les espaces multiples
    texte = re.sub(r'\s+', ' ', texte).strip()
    # Supprimer les mots inutiles
    mots = texte.split()  # Divise le texte en mots avec split (pas de tokenisation)
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots)  # Recombine les mots filtrés


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Liste des colonnes textuelles
colonnes_textuelles = [
    "métier", "appellations", "identité", "compétences", 
]

# Vérifiez quelles colonnes existent dans le DataFrame
colonnes_existe = [col for col in colonnes_textuelles if col in data_cleaned.columns]


for col in colonnes_existe:
    data_cleaned[col] = data_cleaned[col].fillna('').apply(nettoyer_colonne_avance)

In [28]:
data_cleaned = data_cleaned.fillna('')

In [29]:
data_cleaned.head()

,métier,appellations,identité,compétences
0,conception développement produits assurances c...,chargé chargée produit assurances chef produit...,définition conçoit adapte nouveaux produits as...,compétences base savoir faire analyser données...
1,conseil clientèle assurances c1102,agent commercial agente commerciale assurances...,définition informe conseille particuliers entr...,compétences base savoir faire analyser besoins...
2,courtage assurances c1103,agent général agente générale assurances court...,définition accompagne clientèle particuliers p...,compétences base savoir faire mettre oeuvre ac...
3,direction exploitation assurances c1104,chargé chargée agence assurances chargé chargé...,définition organise planification activité ges...,compétences base savoir faire définir stratégi...
4,études actuarielles assurances c1105,actuaire confirmé sénior actuaire junior charg...,définition evalue court moyen long terme renta...,compétences base savoir faire proposer axes am...


In [30]:
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

# Appliquer la tokenisation à chaque cellule de toutes les colonnes
data_cleaned = data_cleaned.apply(lambda col: col.apply(word_tokenize))

# Vérifier les résultats
print(data_cleaned.head())

                                              métier  \
0  [conception, développement, produits, assuranc...   
1            [conseil, clientèle, assurances, c1102]   
2                      [courtage, assurances, c1103]   
3       [direction, exploitation, assurances, c1104]   
4          [études, actuarielles, assurances, c1105]   

                                        appellations  \
0  [chargé, chargée, produit, assurances, chef, p...   
1  [agent, commercial, agente, commerciale, assur...   
2  [agent, général, agente, générale, assurances,...   
3  [chargé, chargée, agence, assurances, chargé, ...   
4  [actuaire, confirmé, sénior, actuaire, junior,...   

                                            identité  \
0  [définition, conçoit, adapte, nouveaux, produi...   
1  [définition, informe, conseille, particuliers,...   
2  [définition, accompagne, clientèle, particulie...   
3  [définition, organise, planification, activité...   
4  [définition, evalue, court, moyen, long, te

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Asma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [31]:
from nltk.stem.snowball import FrenchStemmer

# Initialiser le stemmer pour le français
stemmer = FrenchStemmer()

# Exemple d'application du stemming sur des données tokenisées
data_cleaned = data_cleaned.applymap(
    lambda x: [stemmer.stem(word) for word in x] if isinstance(x, list) else x
)

# Vérifier les résultats
print(data_cleaned.head())

                                       métier  \
0  [concept, développ, produit, assur, c1101]   
1           [conseil, clientel, assur, c1102]   
2                     [courtag, assur, c1103]   
3             [direct, exploit, assur, c1104]   
4             [étud, actuariel, assur, c1105]   

                                        appellations  \
0  [charg, charg, produit, assur, chef, produit, ...   
1  [agent, commercial, agent, commercial, assur, ...   
2  [agent, général, agent, général, assur, courti...   
3  [charg, charg, agenc, assur, charg, charg, bur...   
4  [actuair, confirm, sénior, actuair, junior, ch...   

                                            identité  \
0  [définit, conçoit, adapt, nouveau, produit, as...   
1  [définit, inform, conseil, particuli, entrepri...   
2  [définit, accompagn, clientel, particuli, prof...   
3  [définit, organis, planif, activ, gestion, adm...   
4  [définit, evalu, court, moyen, long, term, ren...   

                               

C:\Users\Asma\AppData\Local\Temp\ipykernel_14224\3638393854.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_cleaned = data_cleaned.applymap(


In [33]:
#Vectorisation
#TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer


# Dictionnaire pour stocker les résultats vectorisés
vecteurs = {}

# Vectorisation des colonnes
tfidf_vectorizer = TfidfVectorizer()

for col in data_cleaned:
    # Appliquer TF-IDF à chaque colonne
    vecteurs[col] = tfidf_vectorizer.fit_transform(data_cleaned[col].astype(str))
    print(f"Colonne '{col}' vectorisée avec {len(tfidf_vectorizer.get_feature_names_out())} termes.")

# Combiner tous les vecteurs dans une seule matrice
from scipy.sparse import hstack

vecteurs_combines = hstack(list(vecteurs.values()))

# Afficher la dimension de la matrice finale
print(f"Dimensions de la matrice combinée : {vecteurs_combines.shape}")

Colonne 'métier' vectorisée avec 171 termes.
Colonne 'appellations' vectorisée avec 585 termes.
Colonne 'identité' vectorisée avec 609 termes.
Colonne 'compétences' vectorisée avec 843 termes.
Dimensions de la matrice combinée : (70, 2208)
